# Big Project: Fantasy Premier League (FPL) Points Predictor
## Data Rules Everything Around Me (DREAM) TEAM - Fall 2024 - CME 538
## Feras Abdulla - Maha Fakhroo - Syed Shahid Hossaini - Eric Guan
-----------------------------------------------------------------------------------------------------------------------------------------------
# Data Wrangling
-----------------------------------------------------------------------------------------------------------------------------------------------

### Objective:
The purpose of this Python script is to collect, align, and merge data from different sources and covering different football seasons into a single `'master'` file. This new `'master'` file will be used for data cleaning/processing, exploratory data analysis (EDA), and creating visualizations (see EDA_Visuals.ipynb), and a filtered version will be split into training and testing subsets to create and evaluate an ML points predictor model (see ML_Points_Predictor.ipynb).

### Data Sources:
We have relied on 3 main sources of data:

1) `Vaastav's 'Fantasy-Premier-League' Github Repository`: One of the most comprehensive, publicly available FPL repositories. Contains `downloadable csv files` that track summary statistics for the whole season, fixture information, team data, gameweek-specific metrics, and prior historical player data. The information has been collated by season, by player, and by gameweek. We will utilize the `'merged_gw'` files available on this repository, which contain gameweek-by-gameweeek stats for all players in a single file for a single season. 

`Reference`: 
Anand, V. (2022). FPL historical dataset. Retrieved November 2024, from https://github.com/vaastav/Fantasy-Premier-League/

2) `Official FPL API`: Contains overview statistics and detailed player data for the current season (2024-2025). The overview statistics can be accessed via the Official FPL website's `bootstrap-static` API, while the detailed player info can be obtained using that same API's element summary endpoint. 

`Reference`:
Fantasy Premier League. (n.d.). FPL API. Retrieved November 2024, from https://fantasy.premierleague.com/api/ 

3) `Understat API`: Provides access to advanced football statistics and metrics for teams, players, and matches. It can be used to retrieve match details, player statistics, team performance, and seasonal summaries. This API will be used to extract expected goals (xG), expected assists (xA), and expected goal involvements (xGI) for each player in earlier seasons, since the `'merged_gw'` files from the `Vaastav` repository do not provide that information prior to the 2022-2023 season.

`Reference`:
Understat. (n.d.). Understat API. Retrieved November 2024, from https://understat.com/

### Import Packages
Imports essential Python libraries for data wrangling. These will be used to extract data from API endpoints, wrangle information across files, and align and publish a filtered data set.

In [7]:
import requests
import pandas as pd
import unicodedata
import time
import os
import asyncio
from understat import Understat
import aiohttp
from datetime import timedelta
import shutil

### Extracting Data from the Official FPL API

First, we will obtain data from the FPL API at the bootstrap-static endpoint and organize it into structured dataframes. We will extract the following key categories: 
1) Player details (elements)
2) Positions (element_types) 
3) Teams (teams) 
4) Gameweeks (events)

Columns in the elements DataFrame are renamed for better clarity and merged with team and position DataFrames to add team names and player positions. After cleaning and merging redundant columns, the final DataFrame (elements_final) provides detailed player information, including team and position details. The cleaned player data is also saved as a CSV file (fpl_elements.csv) for further use.

In [8]:
# Fetch the data from the FPL API
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
data = response.json()

# Organize the main data categories into DataFrames
elements_df = pd.DataFrame(data['elements'])  # Player statistics and details
element_types_df = pd.DataFrame(data['element_types'])  # Player positions
teams_df = pd.DataFrame(data['teams'])  # Team information
events_df = pd.DataFrame(data['events'])  # Gameweeks
game_settings_df = pd.DataFrame([data['game_settings']])  # Game settings
phases_df = pd.DataFrame(data['phases'])  # Season phases

# Save file in 'Official FPL API Files' Folder
elements_df.to_csv("../Official FPL API Files/fpl_elements.csv", index=False)

# Renaming columns in elements_df to more descriptive names for convenience
elements_df = elements_df.rename(columns={
    'team': 'Team ID',
    'element_type': 'Position ID',
    'total_points': 'Total Points',
    'now_cost': 'Current Price',
    'selected_by_percent': 'Selected By (%)'
})

# Merge elements_df with teams_df to get team names for players
elements_with_teams = elements_df.merge(teams_df[['id', 'name']], how='left', left_on='Team ID', right_on='id')
elements_with_teams.rename(columns={'name': 'Team Name'}, inplace=True)

# Drop redundant 'id' column only if it exists
if 'id' in elements_with_teams.columns:
    elements_with_teams.drop(columns=['id'], inplace=True)

# Merge elements_with_teams with element_types_df to get position names for players
elements_final = elements_with_teams.merge(element_types_df[['id', 'singular_name']], how='left', left_on='Position ID', right_on='id')
elements_final.rename(columns={'singular_name': 'Position Name'}, inplace=True)

# Drop redundant 'id' column from element_types_df merge only if it exists
if 'id' in elements_final.columns:
    elements_final.drop(columns=['id'], inplace=True)

# Display the final DataFrame structure
print(elements_final.head())


   chance_of_playing_next_round  chance_of_playing_this_round    code  \
0                           0.0                           0.0  438098   
1                         100.0                         100.0  205651   
2                         100.0                         100.0  226597   
3                         100.0                         100.0  219847   
4                           0.0                           0.0  463748   

   cost_change_event  cost_change_event_fall  cost_change_start  \
0                  0                       0                 -1   
1                  0                       0                 -2   
2                  0                       0                  1   
3                  0                       0                  0   
4                  0                       0                  0   

   cost_change_start_fall  dreamteam_count  Position ID ep_next  ...  \
0                       1                0            3     0.0  ...   
1             

Now, let's proceed with getting detailed player data rather than just seasonal overview statistics. Here, we intend to build a comprehensive dataset of player performance from the official FPL API across gameweeks focusing on the ongoing, 2024-2025 season.

The columns player IDs, first names, and last names are extracted from fpl_elements.csv to facilitate merging with gameweek data later. The element-summary endpoint of the FPL API will be engaged to store data for every player's performance across all gameweeks using each player's unique ID. For each gameweek entry, the code adds the player's ID, first name, and last name to the record before appending it to the list. After collecting data for all players, the list is converted into a DataFrame (gameweek_df). The first few rows of the processed DataFrame are presented.

In [9]:
# Load the elements data to get player names and IDs
elements_df = pd.read_csv("../Official FPL API Files/fpl_elements.csv")

# Select only the necessary columns for merging later
elements_df = elements_df[['id', 'first_name', 'second_name']]

# Initialize an empty list to collect data for each player per gameweek
all_gameweek_data = []

# Loop through each player's ID in elements_df
for _, row in elements_df.iterrows():
    player_id = row['id']
    first_name = row['first_name']
    last_name = row['second_name']
    
    # Fetch gameweek data for each player
    url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        player_data = response.json()
        
        # Extract each gameweek's data and add it to our list
        for gameweek in player_data['history']:
            # Add player ID, first name, and last name to each gameweek record
            gameweek['player_id'] = player_id
            gameweek['first_name'] = first_name
            gameweek['last_name'] = last_name
            all_gameweek_data.append(gameweek)
    
    # Optional: Sleep to avoid rate limiting
    time.sleep(0.5)

# Convert the collected gameweek data into a DataFrame
gameweek_df = pd.DataFrame(all_gameweek_data)

# Rename columns for clarity where applicable
column_renames = {
    'round': 'Gameweek',
    'team_h_score': 'Goals_for_Home_Team',
    'team_a_score': 'Goals_for_Away_Team',
    'bonus': 'Bonus_Points',
    'value': 'Market_Price',
    'ict_index': 'Influence_Creativity_Threat_Index',
    'influence': 'Influence_Score',
    'creativity': 'Creativity Score',
    'threat': 'Threat Score',
}

gameweek_df.rename(columns=column_renames, inplace=True)

# Display the first few rows to confirm
print(gameweek_df.head())


   element  fixture  opponent_team  total_points  was_home  \
0        1        2             20             0      True   
1        1       11              2             0     False   
2        1       21              5             0      True   
3        1       39             18             0     False   
4        1       47             13             0     False   

           kickoff_time  Goals_for_Home_Team  Goals_for_Away_Team  Gameweek  \
0  2024-08-17T14:00:00Z                  2.0                  0.0         1   
1  2024-08-24T16:30:00Z                  0.0                  2.0         2   
2  2024-08-31T11:30:00Z                  1.0                  1.0         3   
3  2024-09-15T13:00:00Z                  0.0                  1.0         4   
4  2024-09-22T15:30:00Z                  2.0                  2.0         5   

   minutes  ...  expected_goal_involvements  expected_goals_conceded  \
0        0  ...                        0.00                     0.00   
1       

Now, we can convert all dataframe to CSV files and save them in the Official API Files folder:

`'fpl_element_types.csv'`: A CSV file containing details about player positions, such as their IDs and names (e.g., Goalkeeper, Forward).

`'fpl_teams.csv'`: A CSV file with team information, including team IDs, names, and other attributes like strength and performance.

`'fpl_events.csv'`: A CSV file with gameweek information, including details like deadlines, fixture results, and gameweek IDs.

`'fpl_game_settings.csv'`: A CSV file with overall game settings, such as scoring rules, transfer rules, and other configuration details for the season.

`'fpl_phases.csv'`: A CSV file containing data about the phases of the season (e.g., first half, second half) and their respective start and end dates.

`'FPL API 2024-2025 Season.csv'`: A CSV file with detailed player performance data for the 2024-2025 season, including gameweek-by-gameweek stats for each player (e.g., points, goals, assists, and team results).

`'fpl_elements.csv'`: A CSV file with player-level data, including player IDs, names, teams, positions, total points, and selected percentages (this CSV file was created in the previous step).

In [10]:
# Create csv files

# Save each DataFrame to CSV for further analysis

element_types_df.to_csv("../Official FPL API Files/fpl_element_types.csv", index=False)
teams_df.to_csv("../Official FPL API Files/fpl_teams.csv", index=False)
events_df.to_csv("../Official FPL API Files/fpl_events.csv", index=False)
game_settings_df.to_csv("../Official FPL API Files/fpl_game_settings.csv", index=False)
phases_df.to_csv("../Official FPL API Files/fpl_phases.csv", index=False)
gameweek_df.to_csv("../Official FPL API Files/FPL API 2024-2025 Season.csv", index=False)

### Extracting Data from the Understat API

Now, we will fetch player data from the Understat API to compile gameweek-level match statistics. An asynchronous HTTP session is created to enable interaction with the Understat API. Player-level data, like ID and aggregate stats, is retrieved for a specified season. The code loops through the list of players retrieved for the season and prints out a sample match structure for the first player in the list for 2020. 

The goal here is to learn the type of data available through the Understat API, in order to determine which elemnents should be extracted and added to our analysis to supplement the bulk of the data we will use, which will come from the `Vaastav` repository.

In [11]:
async def fetch_understat_data(season):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        
        # Fetch player data for the specified season in the Premier League
        players_data = await understat.get_league_players("epl", season)
        
        # Create an empty list to hold the player gameweek data
        player_gameweek_data = []

        for player in players_data:
            player_id = player['id']
            player_name = player['player_name']
            
            # Fetch player matches (gameweek data) for each player in the season
            matches = await understat.get_player_matches(player_id)
            
            # Check the structure of a single match
            if matches:
                print(f"Sample match data for {player_name} in {season}:", matches[0])
                break

# Run this code to view the structure of a match data item
await fetch_understat_data("2020")

Sample match data for Harry Kane in 2020: {'goals': '3', 'shots': '6', 'xG': '2.4590251445770264', 'time': '90', 'position': 'FW', 'h_team': 'Bayern Munich', 'a_team': 'Augsburg', 'h_goals': '3', 'a_goals': '0', 'date': '2024-11-22', 'id': '27834', 'season': '2024', 'roster_id': '687931', 'xA': '0', 'assists': '0', 'key_passes': '0', 'npg': '1', 'npxG': '0.9434716701507568', 'xGChain': '1.0221545696258545', 'xGBuildup': '0.07868286967277527'}


We can see that Understat provides us with expected goals and expected assists, which are two metrics that are missing from the `Vaastav` files prior to the 2022-2023 season. We will want to extract these for our analysis.

Now, we will collect these advanced football statistics from the Understat API for multiple Premier League seasons. We will asynchronously retrieve player-level data and match-level statistics, including expected goals (xG), expected assists (xA), and their combined metric, expected goal involvements (xGI), for each player in each gameweek. The collected data is organized into a DataFrame and saved as CSV files, one for each season. The script uses asynchronous processing for efficiency and includes error handling and delays to avoid overloading the server.

`NOTE`: This will take 15-20 minutes.

In [12]:
async def fetch_understat_data(season):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        
        # Fetch player data for the specified season in the Premier League
        players_data = await understat.get_league_players("epl", season)
        
        # Create an empty list to hold the player gameweek data
        player_gameweek_data = []

        for player in players_data:
            player_id = player['id']
            player_name = player['player_name']
            
            # Fetch player matches (gameweek data) for each player in the season
            matches = await understat.get_player_matches(player_id)
            
            # Extract xG, xA, and calculate expected goal involvements
            for match in matches:
                if match['season'] == season:
                    gameweek_data = {
                        "player_name": player_name,
                        "date": match['date'],
                        "xG": float(match['xG']),
                        "xA": float(match['xA']),
                        "expected_goal_involvements": float(match['xG']) + float(match['xA'])
                    }
                    player_gameweek_data.append(gameweek_data)
        
        # Convert the list to a DataFrame for easy manipulation
        return pd.DataFrame(player_gameweek_data)

# Fetch data for the seasons from 2016/17 to 2021/22
async def main():
    seasons = ["2016", "2017", "2018", "2019", "2020", "2021"]
    for season in seasons:
        print(f"Fetching data for the {season}/{str(int(season) + 1)[-2:]} season...")
        try:
            season_data = await fetch_understat_data(season)
            season_data.to_csv(f"../Understat API Files/understat_{season}_{str(int(season) + 1)[-2:]}.csv", index=False)
            print(f"Data for {season}/{str(int(season) + 1)[-2:]} saved successfully.")
        except Exception as e:
            print(f"Failed to fetch data for {season}/{str(int(season) + 1)[-2:]}. Error: {e}")
        
        # Wait a few seconds between each season request to avoid overloading the server
        await asyncio.sleep(5)  # Adjust this delay as needed

# Run the asynchronous main function
await main()



Fetching data for the 2016/17 season...
Data for 2016/17 saved successfully.
Fetching data for the 2017/18 season...
Data for 2017/18 saved successfully.
Fetching data for the 2018/19 season...
Data for 2018/19 saved successfully.
Fetching data for the 2019/20 season...
Data for 2019/20 saved successfully.
Fetching data for the 2020/21 season...
Data for 2020/21 saved successfully.
Fetching data for the 2021/22 season...
Data for 2021/22 saved successfully.


## Combining Understat API and Vaastav Data

We will now combine Understat data (xG, xA, xGI) with Vaastav's merged_gw files to create a comprehensive dataset for each Premier League season. Player names are normalized in both datasets by removing special characters and suffixes for consistency. Gameweeks are assigned to Understat data based on match dates, grouping matches into 7-day periods, ensuring alignment with gameweek structures in the Vaastav dataset. The datasets are then merged by player name and gameweek, with Understat's metrics (xG, xA, xGI) being matched to the most chronologically-appropriate Vaastav entries. Any unmatched rows in the merged dataset (e.g., where Understat lacks data for a player or gameweek) result in NaN values. The combined data is saved as a CSV for each season, allowing for seamless integration of advanced analytics with official FPL data.

In [14]:
def normalize_name(name):
    """Normalize player names by removing special characters and suffixes."""
    name = unicodedata.normalize('NFKD', name)
    name = ''.join(c for c in name if not unicodedata.combining(c))
    if "_" in name:
        # Remove suffix numbers, e.g., '_191'
        name = '_'.join(name.split('_')[:2])
    return name.replace("_", " ")  # Replace underscores with spaces

def assign_gameweeks(understat_df):
    """Assign gameweeks based on the date column in the understat data."""
    # Sort dates chronologically
    understat_df['date'] = pd.to_datetime(understat_df['date'])
    understat_df = understat_df.sort_values(by='date').reset_index(drop=True)

    # Initialize gameweek assignment
    gameweeks = []
    current_gw_start = understat_df['date'].iloc[0]
    current_gw_end = current_gw_start + timedelta(days=6)
    current_gw = 1

    for game_date in understat_df['date']:
        if game_date > current_gw_end:
            # Start a new gameweek
            current_gw_start = game_date
            current_gw_end = current_gw_start + timedelta(days=6)
            current_gw += 1
        gameweeks.append(current_gw)

    understat_df['gameweek'] = gameweeks
    return understat_df

def merge_understat_with_merged_gw(season_start, season_end):
    """Merge understat data with merged_gw data for a given season."""
    understat_file = f"../Understat API Files/understat_20{season_start}_{season_end}.csv"
    merged_gw_file = f"../Vaastav Repository Files (CSV Downloads)/merged_gw_{season_start[-2:]}{season_end[-2:]}.csv"
    output_file = f"../Combined Files/20{season_start}-20{season_end} season data.csv"

    # Load the data with proper encoding
    understat_df = pd.read_csv(understat_file, encoding='ISO-8859-1')
    merged_gw_df = pd.read_csv(merged_gw_file, encoding='ISO-8859-1')

    # Normalize names in both datasets
    merged_gw_df['name'] = merged_gw_df['name'].apply(normalize_name)
    understat_df['player_name'] = understat_df['player_name'].apply(normalize_name)

    # Assign gameweeks based on understat dates
    understat_df = assign_gameweeks(understat_df)

    # Convert kickoff_time to date format (YYYY-MM-DD)
    merged_gw_df['kickoff_date'] = pd.to_datetime(merged_gw_df['kickoff_time']).dt.date
    understat_df['date'] = understat_df['date'].dt.date

    # Merge data on name and gameweek
    combined_df = pd.merge(
        merged_gw_df,
        understat_df,
        left_on=['name', 'kickoff_date'],
        right_on=['player_name', 'date'],
        how='left'
    )

    # Drop redundant columns
    combined_df.drop(['player_name', 'date', 'kickoff_date'], axis=1, inplace=True)

    # Save the combined data
    combined_df.to_csv(output_file, index=False, na_rep='NaN')
    print(f"Saved combined data for 20{season_start}-20{season_end} season to {output_file}")

    # Report the number of NaN rows in xA, xG, and xGI columns
    nan_counts = combined_df[['xA', 'xG', 'expected_goal_involvements']].isna().sum()
    print(f"Season 20{season_start}-20{season_end}: NaN counts - xA: {nan_counts['xA']}, xG: {nan_counts['xG']}, xGI: {nan_counts['expected_goal_involvements']}")


# List of seasons
seasons = [
    ("16", "17"),
    ("17", "18"),
    ("18", "19"),
    ("19", "20"),
    ("20", "21"),
    ("21", "22"),
]

# Process each season
for season_start, season_end in seasons:
    merge_understat_with_merged_gw(season_start, season_end)


Saved combined data for 2016-2017 season to ../Combined Files/2016-2017 season data.csv
Season 2016-2017: NaN counts - xA: 15178, xG: 15178, xGI: 15178
Saved combined data for 2017-2018 season to ../Combined Files/2017-2018 season data.csv
Season 2017-2018: NaN counts - xA: 13966, xG: 13966, xGI: 13966
Saved combined data for 2018-2019 season to ../Combined Files/2018-2019 season data.csv
Season 2018-2019: NaN counts - xA: 13814, xG: 13814, xGI: 13814
Saved combined data for 2019-2020 season to ../Combined Files/2019-2020 season data.csv
Season 2019-2020: NaN counts - xA: 13562, xG: 13562, xGI: 13562
Saved combined data for 2020-2021 season to ../Combined Files/2020-2021 season data.csv
Season 2020-2021: NaN counts - xA: 15749, xG: 15749, xGI: 15749
Saved combined data for 2021-2022 season to ../Combined Files/2021-2022 season data.csv
Season 2021-2022: NaN counts - xA: 16899, xG: 16899, xGI: 16899


Note that we merged the Understat and Vaastav data only through the 2021-2022 season. We do this because later seasons (22/23, 23/24, and the ongoing 24/25) already include advanced metrics (xG, xA, xGI) in the Vaastav and official FPL API data, so the existing files can already be considered 'combined' and do not need further alterations.

Below, we will copy and rename these final 3 files into the same directory as the files we just combined using both Vaastav and Understat data (D.R.E.A.M./Combined Files).

In [15]:
# Define source and destination directories
source_dir = "../Vaastav Repository Files (CSV Downloads)"
destination_dir = "../Combined Files"

# List of source and destination file names
files_to_copy = [
    ("merged_gw_2223.csv", "2022-2023 season data.csv"),
    ("merged_gw_2324.csv", "2023-2024 season data.csv"),
    ("merged_gw_2425.csv", "2024-2025 season data.csv"),
]

# Iterate through the list and copy/rename files
for source_file, destination_file in files_to_copy:
    source_path = os.path.join(source_dir, source_file)
    destination_path = os.path.join(destination_dir, destination_file)

    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied and renamed: {source_path} -> {destination_path}")
    except FileNotFoundError:
        print(f"File not found: {source_path}")
    except Exception as e:
        print(f"An error occurred while processing {source_path}: {e}")


Copied and renamed: ../Vaastav Repository Files (CSV Downloads)/merged_gw_2223.csv -> ../Combined Files/2022-2023 season data.csv
Copied and renamed: ../Vaastav Repository Files (CSV Downloads)/merged_gw_2324.csv -> ../Combined Files/2023-2024 season data.csv
Copied and renamed: ../Vaastav Repository Files (CSV Downloads)/merged_gw_2425.csv -> ../Combined Files/2024-2025 season data.csv


## Converting into DataFrames and Creating a Master File

First, we will create a dataframe for each combined file.

In [16]:
s1617 = pd.read_csv('../Combined Files/2016-2017 season data.csv')
s1718 = pd.read_csv('../Combined Files/2017-2018 season data.csv')
s1819 = pd.read_csv('../Combined Files/2018-2019 season data.csv')
s1920 = pd.read_csv('../Combined Files/2019-2020 season data.csv')
s2021 = pd.read_csv('../Combined Files/2020-2021 season data.csv')
s2122 = pd.read_csv('../Combined Files/2021-2022 season data.csv')
s2223 = pd.read_csv('../Combined Files/2022-2023 season data.csv')
s2324 = pd.read_csv('../Combined Files/2023-2024 season data.csv')
s2425 = pd.read_csv('../Combined Files/2024-2025 season data.csv')

Next, we will create a list of the dataframes and iterate through them to align certain columns that may be referred to differently across the Understat and Vaastav files. xA will be called expected_assists, xG will be called expected_goals, ict_index will be called Influence_Creativity_Threat_Index, and the gameweek column will be dropped (because this data can be extracted from the datetime column 'kickoff time') across all the dataframes.

In [17]:
season_list = [s1617, s1718, s1819, s1920, s2021, s2122, s2223, s2324, s2425]

for df in season_list:
    if 'xA' in df.columns:
        df.rename(columns={'xA': 'expected_assists'}, inplace=True)
    if 'xG' in df.columns:
        df.rename(columns={'xG': 'expected_goals'}, inplace=True)
    if 'ict_index' in df.columns:
        df.rename(columns={'ict_index': 'Influence_Creativity_Threat_Index'}, inplace=True)
    if 'gameweek' in df.columns:
        df.drop(columns=['gameweek'], inplace=True)

Now, we need to examine the columns that are shared across the dataframes vs the ones unique to certain seasons.

In [18]:
# Dictionary to store column names for each DataFrame
df_columns = {}

# Extract column names for each DataFrame
for i, df in enumerate(season_list):
    try:
        season_name = f"Season {16 + i}-{17 + i}"  # Create a season label
        df_columns[season_name] = set(df.columns)
    except Exception as e:
        print(f"An error occurred while processing {season_name}: {e}")

# Find columns shared by all DataFrames (intersection of all column sets)
shared_columns = set.intersection(*df_columns.values())

# Print shared columns
print("Shared columns across all DataFrames:")
print(shared_columns)
print("\n")

# Identify and print unique columns for each DataFrame
for season, columns in df_columns.items():
    unique_columns = columns - shared_columns
    print(f"Season: {season}")
    print(f"Unique columns: {unique_columns if unique_columns else 'None'}")
    print("\n")

Shared columns across all DataFrames:
{'influence', 'opponent_team', 'goals_conceded', 'yellow_cards', 'own_goals', 'was_home', 'value', 'Influence_Creativity_Threat_Index', 'saves', 'transfers_balance', 'kickoff_time', 'clean_sheets', 'total_points', 'red_cards', 'round', 'bonus', 'element', 'minutes', 'goals_scored', 'team_a_score', 'penalties_missed', 'expected_assists', 'GW', 'bps', 'threat', 'creativity', 'selected', 'transfers_in', 'penalties_saved', 'expected_goal_involvements', 'name', 'transfers_out', 'assists', 'team_h_score', 'expected_goals', 'fixture'}


Season: Season 16-17
Unique columns: {'dribbles', 'target_missed', 'big_chances_missed', 'offside', 'clearances_blocks_interceptions', 'kickoff_time_formatted', 'id', 'loaned_out', 'tackled', 'loaned_in', 'ea_index', 'tackles', 'fouls', 'completed_passes', 'errors_leading_to_goal', 'penalties_conceded', 'key_passes', 'recoveries', 'big_chances_created', 'open_play_crosses', 'winning_goals', 'attempted_passes', 'errors_lead

Some important observations:

1) We notice that all dataframes share several columns common across all seasons, especially fundamental data like player name, fixture, home vs away information, and basic appearance and scoring metrics. 
2) 2019/2020 season has the smallest number of columns, mainly due to the initial impact of the COVID-19 pandemic and the distortion in the PL schedule.
3) 2016/2017, 2017/2018, and 2018/2019 are aligned in terms of columns shared, while 2020/2021 to 2024/2025 are also aligned.

To ensure complete consistency in the 2020/2021 to 2024/2025 seasons, we will drop the `'expected_goals_conceded'` and `'starts'` columns, which are absent from seasons 2020/2021 and 2021/2022.

In [19]:
for df in [s2223, s2324, s2425]:
    df.drop(columns=['expected_goals_conceded', 'starts'], inplace=True)


Now, let's check again to see what columns are shared by seasons 2020/2021 to 2024/2025. They should have all columns in common now.

In [20]:
# Dictionary to store column names for each DataFrame
df_columns = {}
list2 = [s2021, s2122, s2223, s2324, s2425]

# Extract column names for each DataFrame
for i, df in enumerate(list2):
    try:
        season_name = f"Season {20 + i}-{21 + i}"  # Create a season label
        df_columns[season_name] = set(df.columns)
    except Exception as e:
        print(f"An error occurred while processing {season_name}: {e}")

# Find columns shared by all DataFrames (intersection of all column sets)
shared_columns = set.intersection(*df_columns.values())

# Print shared columns
print("Shared columns across all DataFrames:")
print(shared_columns)
print("\n")

# Identify and print unique columns for each DataFrame
for season, columns in df_columns.items():
    unique_columns = columns - shared_columns
    print(f"Season: {season}")
    print(f"Unique columns: {unique_columns if unique_columns else 'None'}")
    print("\n")

Shared columns across all DataFrames:
{'influence', 'opponent_team', 'goals_conceded', 'yellow_cards', 'own_goals', 'was_home', 'value', 'Influence_Creativity_Threat_Index', 'saves', 'transfers_balance', 'kickoff_time', 'team', 'clean_sheets', 'total_points', 'red_cards', 'round', 'position', 'bonus', 'element', 'expected_assists', 'goals_scored', 'minutes', 'penalties_missed', 'team_a_score', 'GW', 'bps', 'threat', 'creativity', 'selected', 'transfers_in', 'penalties_saved', 'expected_goal_involvements', 'name', 'transfers_out', 'assists', 'team_h_score', 'expected_goals', 'fixture', 'xP'}


Season: Season 20-21
Unique columns: None


Season: Season 21-22
Unique columns: None


Season: Season 22-23
Unique columns: None


Season: Season 23-24
Unique columns: None


Season: Season 24-25
Unique columns: None




We will now combine the individual seasonal dataframes into a `'master'` dataframe.

In [21]:
# Combine all DataFrames into a single DataFrame
master = pd.concat(list2, ignore_index=True)

master.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,expected_goals,expected_assists,expected_goal_involvements
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,0,0,0,55,True,0,1,0.392763,0.000000,0.392763
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,0,0,0,50,True,0,1,0.000000,0.000000,0.000000
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,0,0,0,50,True,0,1,NaN,NaN,NaN
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0,0,50,True,0,1,0.000000,0.000000,0.000000
4,Abdoulaye DoucourA©,MID,Everton,1.3,0,0,20,1,44.6,512,...,0,0,0,55,False,0,1,0.000000,0.205708,0.205708


Let's see how large the dataframe is and how many missing values it contains.

In [22]:
print('Shape of master dataframe: ', master.shape)
print('Number of missing values in master dataframe: ', master.isna().any(axis=1).sum())


Shape of master dataframe:  (111920, 39)
Number of missing values in master dataframe:  32648


Now to save this master dataframe into a `'master.csv'` file, on which we will perform EDA, visualizations, and eventually train an ML model.

In [23]:
# Save the combined DataFrame to a CSV file
master.to_csv("../master.csv", index=False)

Let's do the same for seasons 2016/2017 to 2018/2019. We will ignore season 2019/2020 due to its relative dearth of information.

In [24]:
# List of DataFrames to combine
dataframes_old = [s1617, s1718, s1819]

# Combine all DataFrames into a single DataFrame
df_old = pd.concat(dataframes_old, ignore_index=True)

df_old.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,expected_goals,expected_assists,expected_goal_involvements
0,Aaron Cresswell,0,0,0,0,0,0,0,0,0,...,0,0,55,False,0,0,1,NaN,NaN,NaN
1,Aaron Lennon,0,3,0,0,0,6,0,1,2,...,0,0,60,True,0,0,1,0.000000,0.0,0.000000
2,Aaron Ramsey,0,26,0,0,0,5,0,2,22,...,0,0,80,True,0,0,1,0.076822,0.0,0.076822
3,Abdoulaye Doucoure,0,0,0,0,0,0,0,0,0,...,0,0,50,False,0,0,1,NaN,NaN,NaN
4,Abdul Rahman Baba,0,0,0,0,0,0,0,0,0,...,0,0,55,True,0,0,1,NaN,NaN,NaN


In [25]:
print('Shape of old dataframe: ', df_old.shape)
print('Number of missing values in old dataframe: ', df_old.isna().any(axis=1).sum())

Shape of old dataframe:  (67936, 59)
Number of missing values in old dataframe:  42958


There are too many missing values in the older seasons (42958 NaNs out of 67936 rows). Therefore, we will focus our EDA, visuals, and model training on the `'master'` file: Premier League seasons 2020/2021 to 2024/2025.